In [1]:
# Importing libs and turning off warnings
import numpy as np
import warnings
warnings.simplefilter('ignore')

from sklearn.datasets import load_breast_cancer

In [2]:
# Loading dataset
X, y = load_breast_cancer(return_X_y=True)
# Checking shape, looks like 1x30 pixels image
print(X[0].shape)
for i,every in enumerate(y):
    if every==0:
        y[i]=-1

(30,)


In [3]:
# Decided to see how it looks like, would use MPL
from matplotlib import pyplot as plt

In [3]:
# Importing testing stuff and sklearn implementation
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [4]:
# Splitting datasets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=17)

In [12]:
%%time
# Testing runtime on n_runs.
# As far as logregression in sklearn depends on random - running multiple times

n_runs=10
summ = float(0.0)
for i in range(n_runs):
    logreg_itc = LogisticRegression()
    inter = cross_val_score(logreg_itc, X, y, cv=5)
    summ+=np.mean(inter)
summ = summ/float(n_runs)

CPU times: user 389 ms, sys: 223 µs, total: 389 ms
Wall time: 388 ms


In [15]:
# Mean accuracy of sklearn on that task
print('Sklearn Logistic Regression accuracy:',summ)

Sklearn Logistic Regression accuracy: 0.9526741054251635


In [18]:
# Implementing my own class, inherited from BaseEstimator and ClassifierMixin
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import euclidean_distances


class LogReg_SGD(BaseEstimator, ClassifierMixin):

    def __init__(self, learning_rate=0.00000000085, n_iter=100, init='zeros'):
        # init - 'zeros', 'ones' and 'random' - default
        self.learning_rate = learning_rate
        self.n_iter = n_iter
        self.init = init

        
    # Gradient Descent optimization method.
    def SGD(self):
        N = len(self.y_)
        for i in range(self.n_iter):
            error = self.y_ - np.dot(self.X_, self.bias)
            cost = np.dot(error.transpose(), error) / N
            #if i % int(self.n_iter*0.1) == 0:
            #    print('Iteration {} | Cost is {}'.format(i, cost))
            gradient = - 2 * np.dot(self.X_.transpose(), error)
            self.bias -= self.learning_rate * gradient
        return self.bias
    
    
    def fit(self, X, y):
    # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        # Store the classes seen during fit
        self.classes_ = unique_labels(y)

        self.X_ = X
        self.y_ = y
        # Initialization of weights.
        # Here sklearn depends on random in some way
        # I would try several inits: zeros, ones, random and heuristics. Maybe heuristics.
        # Just for fun tried minus ones as well x)
        if self.init=='zeros':
            self.bias = np.zeros(len(X[0]))
        if self.init=='ones':
            self.bias = np.ones(len(X[0]))
        if self.init=='mones':
            self.bias = np.empty(len(X[0]))
            self.bias.fill(-1)
        if self.init=='random':
            self.bias = np.random.rand(len(X[0]))
        
        # Learning process is all about gradient descent in that way.
        # All parameters being transported via self object.
        self.bias = self.SGD()
        
        # Return the classifier
        return self

    
    # Function: R->[0,1)
    # Leads from distance to "probability" for sample to be in 1 class.
    def sigmoid(self, z): return 1.0 / (1 + np.exp(-z))
    
    def predict_prob(self, X):
    # Check is fit had been called
        check_is_fitted(self, ['X_', 'y_'])
    # Input validation
        X = check_array(X)

        prediction = np.dot(X, self.bias)
        for i,every in enumerate(prediction):
            prediction[i] = self.sigmoid(every)
        return prediction
    
    
    def predict(self, X):
        pred = []
        prob = self.predict_prob(X)
        for every in prob:
            if every>=0.5:
                pred.append(1)
            else:
                pred.append(-1)
        return pred

In [19]:
# External stuff for testing below
def SGD(X, y, bias, learning_rate, num_iter):
    N = len(y)
    for i in range(num_iter):
        error = y - np.dot(X, bias)
        cost = np.dot(error.transpose(), error) / N
        if i % int(num_iter/10) == 0:
            print('Iteration {} | Cost is {}'.format(i, cost))
        gradient = - 2 * np.dot(X.transpose(), error)
        bias -= learning_rate * gradient
    return bias

In [31]:
# Used to test how big should LR be.
# Found smthg like optimal at 0.000000000857, with number of iterations about 100.
# Bigger causes Nan in answer
weights = SGD(X,y,np.zeros(30),learning_rate=0.000000000857, num_iter=1000)

Iteration 0 | Cost is 1.0
Iteration 100 | Cost is 0.7381425569916857
Iteration 200 | Cost is 0.6791947283745655
Iteration 300 | Cost is 0.6483280099116778
Iteration 400 | Cost is 0.6257091188818952
Iteration 500 | Cost is 0.6078836132728872
Iteration 600 | Cost is 0.5936068645821517
Iteration 700 | Cost is 0.5820874295385967
Iteration 800 | Cost is 0.572732945231407
Iteration 900 | Cost is 0.5650863746531258


In [7]:
# Comparing efficiency of different initialization methods:
init_ones = LogReg_SGD(init='ones')
init_mones = LogReg_SGD(init='mones')
init_zeros = LogReg_SGD(init='zeros')
init_random = LogReg_SGD(init='random')

In [32]:
%%time

# Running 10 times to exclude influence of randomness
n_runs=10
summ_0 = float(0.0)
summ_1 = float(0.0)
summ_r = float(0.0)

for i in range(n_runs):
    res_0 = cross_val_score(init_zeros, X, y, cv=5)
    summ_0+=np.mean(res_0)
    
    res_1 = cross_val_score(init_ones, X, y, cv=5)
    summ_1+=np.mean(res_1)
    
    res_r = cross_val_score(init_random, X, y, cv=5)
    summ_r+=np.mean(res_0)
    
summ_0 = summ_0/float(n_runs)
summ_1 = summ_1/float(n_runs)
summ_r = summ_r/float(n_runs)

print("Initialization with ones accuracy:", summ_1)
print("Initialization with random accuracy:", summ_r)
print("Initialization with zeros accuracy:", summ_0)

Initialization with ones accuracy: 0.8035552135436708
Initialization with random accuracy: 0.9085186610234708
Initialization with zeros accuracy: 0.9085186610234708
CPU times: user 1.51 s, sys: 1.71 s, total: 3.22 s
Wall time: 810 ms


> Cost function is much better in current case for zero-init method.<br>
> So I would compare it with sklearn.

In [39]:
print("Accuracy of my implementation:", summ_0)
print("Accuracy of Sklearn implem.:", summ)
print("Difference:", summ-summ_0)
print("Number of errors in my LR: ~"+str(int((1-summ_0)*len(y)*0.2)))
print("Number of errors in sklearn: ~"+str(int((1-summ)*len(y)*0.2)))

Accuracy of my implementation: 0.9085186610234708
Accuracy of Sklearn implem.: 0.9526741054251635
Difference: 0.04415544440169272
Number of errors in my LR: ~10
Number of errors in sklearn: ~5
